# Reproduce original VGAE (Kipf & Welling 2016) Cora embeddings

## Run the original training code (corrected to run inside Jupyter)

It trains embeddings using the `adj_train` adjacency, where ~15% of the edges have been removed (10% for a test set, 5% for a validation set).

In [1]:
import time
import os

# Train on CPU (hide GPU) due to memory constraints
os.environ['CUDA_VISIBLE_DEVICES'] = ""

import tensorflow as tf
import numpy as np
import scipy.sparse as sp

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

from gae.optimizer import OptimizerAE, OptimizerVAE
from gae.input_data import load_data
from gae.model import GCNModelAE, GCNModelVAE
from gae.preprocessing import preprocess_graph, construct_feed_dict, sparse_to_tuple, mask_test_edges

In [2]:
# Set options and move to necessary directory for data loading
MODEL_STRING = 'gcn_vae'
os.chdir('../../gae')

In [3]:
# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 32, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 16, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0., 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')

flags.DEFINE_string('model', MODEL_STRING, 'Model string.')
flags.DEFINE_string('dataset', 'citeseer', 'Dataset string.')
flags.DEFINE_integer('features', 1, 'Whether to use features (1) or not (0).')

# Workaround for Jupyter having an '-f' flag in its sys.argv -- this is ignored
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [4]:
model_str = FLAGS.model
dataset_str = FLAGS.dataset

# Load data
adj, features = load_data(dataset_str)

# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train

if FLAGS.features == 0:
    features = sp.identity(features.shape[0])  # featureless

# Some preprocessing
adj_norm = preprocess_graph(adj)

# Define placeholders
placeholders = {
    'features': tf.sparse_placeholder(tf.float32),
    'adj': tf.sparse_placeholder(tf.float32),
    'adj_orig': tf.sparse_placeholder(tf.float32),
    'dropout': tf.placeholder_with_default(0., shape=())
}

num_nodes = adj.shape[0]

features = sparse_to_tuple(features.tocoo())
num_features = features[2][1]
features_nonzero = features[1].shape[0]

In [5]:
# Create model
model = None
if model_str == 'gcn_ae':
    model = GCNModelAE(placeholders, num_features, features_nonzero)
elif model_str == 'gcn_vae':
    model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)

pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

# Optimizer
with tf.name_scope('optimizer'):
    if model_str == 'gcn_ae':
        opt = OptimizerAE(preds=model.reconstructions,
                          labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                      validate_indices=False), [-1]),
                          pos_weight=pos_weight,
                          norm=norm)
    elif model_str == 'gcn_vae':
        opt = OptimizerVAE(preds=model.reconstructions,
                           labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                       validate_indices=False), [-1]),
                           model=model, num_nodes=num_nodes,
                           pos_weight=pos_weight,
                           norm=norm)

In [6]:
def get_roc_score(edges_pos, edges_neg, emb=None):
    if emb is None:
        feed_dict.update({placeholders['dropout']: 0})
        emb = sess.run(model.z_mean, feed_dict=feed_dict)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

In [7]:
# Initialize session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

cost_val = []
acc_val = []
val_roc_score = []

adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

In [8]:
# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(adj_norm, adj_label, features, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})
    # Run single weight update
    outs = sess.run([opt.opt_op, opt.cost, opt.accuracy], feed_dict=feed_dict)

    # Compute average loss
    avg_cost = outs[1]
    avg_accuracy = outs[2]

    roc_curr, ap_curr = get_roc_score(val_edges, val_edges_false)
    val_roc_score.append(roc_curr)

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
          "train_acc=", "{:.5f}".format(avg_accuracy), "val_roc=", "{:.5f}".format(val_roc_score[-1]),
          "val_ap=", "{:.5f}".format(ap_curr),
          "time=", "{:.5f}".format(time.time() - t))

print("Optimization Finished!")

roc_score, ap_score = get_roc_score(test_edges, test_edges_false)
print('Test ROC score: ' + str(roc_score))
print('Test AP score: ' + str(ap_score))

Epoch: 0001 train_loss= 1.73741 train_acc= 0.49796 val_roc= 0.61854 val_ap= 0.64434 time= 2.60381
Epoch: 0002 train_loss= 1.43449 train_acc= 0.46155 val_roc= 0.62835 val_ap= 0.66619 time= 0.37947
Epoch: 0003 train_loss= 1.20442 train_acc= 0.42560 val_roc= 0.64696 val_ap= 0.69361 time= 0.75563
Epoch: 0004 train_loss= 1.02252 train_acc= 0.39231 val_roc= 0.68767 val_ap= 0.73140 time= 0.78977
Epoch: 0005 train_loss= 0.86154 train_acc= 0.37355 val_roc= 0.75783 val_ap= 0.77902 time= 0.97244
Epoch: 0006 train_loss= 0.74621 train_acc= 0.39023 val_roc= 0.79796 val_ap= 0.81930 time= 0.78120
Epoch: 0007 train_loss= 0.67363 train_acc= 0.39526 val_roc= 0.81377 val_ap= 0.83499 time= 0.58651
Epoch: 0008 train_loss= 0.62895 train_acc= 0.39088 val_roc= 0.82371 val_ap= 0.84231 time= 0.56865
Epoch: 0009 train_loss= 0.59811 train_acc= 0.41525 val_roc= 0.84294 val_ap= 0.85963 time= 0.40663
Epoch: 0010 train_loss= 0.57273 train_acc= 0.41655 val_roc= 0.85183 val_ap= 0.86229 time= 0.44762
Epoch: 0011 train_lo

Epoch: 0085 train_loss= 0.43604 train_acc= 0.52731 val_roc= 0.89062 val_ap= 0.91097 time= 0.58285
Epoch: 0086 train_loss= 0.43626 train_acc= 0.52722 val_roc= 0.89057 val_ap= 0.91155 time= 0.64926
Epoch: 0087 train_loss= 0.43571 train_acc= 0.52798 val_roc= 0.89064 val_ap= 0.91182 time= 0.84282
Epoch: 0088 train_loss= 0.43559 train_acc= 0.52833 val_roc= 0.89053 val_ap= 0.91123 time= 0.61343
Epoch: 0089 train_loss= 0.43561 train_acc= 0.52818 val_roc= 0.89045 val_ap= 0.91106 time= 0.74100
Epoch: 0090 train_loss= 0.43522 train_acc= 0.52788 val_roc= 0.89080 val_ap= 0.91111 time= 0.67112
Epoch: 0091 train_loss= 0.43524 train_acc= 0.52839 val_roc= 0.89053 val_ap= 0.91087 time= 0.59541
Epoch: 0092 train_loss= 0.43499 train_acc= 0.52914 val_roc= 0.89051 val_ap= 0.91058 time= 0.40283
Epoch: 0093 train_loss= 0.43461 train_acc= 0.52887 val_roc= 0.89084 val_ap= 0.91105 time= 0.38043
Epoch: 0094 train_loss= 0.43461 train_acc= 0.52909 val_roc= 0.89057 val_ap= 0.91051 time= 0.37621
Epoch: 0095 train_lo

Epoch: 0169 train_loss= 0.42719 train_acc= 0.54003 val_roc= 0.89293 val_ap= 0.91716 time= 1.22417
Epoch: 0170 train_loss= 0.42722 train_acc= 0.54079 val_roc= 0.89255 val_ap= 0.91731 time= 0.89257
Epoch: 0171 train_loss= 0.42734 train_acc= 0.53958 val_roc= 0.89356 val_ap= 0.91777 time= 0.87076
Epoch: 0172 train_loss= 0.42726 train_acc= 0.54009 val_roc= 0.89309 val_ap= 0.91729 time= 0.80169
Epoch: 0173 train_loss= 0.42712 train_acc= 0.54057 val_roc= 0.89239 val_ap= 0.91701 time= 0.71421
Epoch: 0174 train_loss= 0.42689 train_acc= 0.54066 val_roc= 0.89388 val_ap= 0.91809 time= 0.59078
Epoch: 0175 train_loss= 0.42698 train_acc= 0.53990 val_roc= 0.89324 val_ap= 0.91753 time= 0.53998
Epoch: 0176 train_loss= 0.42686 train_acc= 0.54112 val_roc= 0.89319 val_ap= 0.91768 time= 0.61857
Epoch: 0177 train_loss= 0.42668 train_acc= 0.54222 val_roc= 0.89258 val_ap= 0.91734 time= 0.78818
Epoch: 0178 train_loss= 0.42711 train_acc= 0.54071 val_roc= 0.89323 val_ap= 0.91757 time= 0.79231
Epoch: 0179 train_lo

## Plot the adjacency reconstruction

In [ ]:
import scipy
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Compute embeddings
feed_dict = construct_feed_dict(adj_norm, adj_label, features, placeholders)
feed_dict.update({placeholders['dropout']: 0})
emb = sess.run(model.z_mean, feed_dict=feed_dict)
adj_pred = np.dot(emb, emb.T)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(3 * 50, 50))
im1 = ax1.imshow(adj_train.toarray())
plt.colorbar(im1, ax=ax1)
im2 = ax2.imshow(scipy.special.expit(adj_pred))
plt.colorbar(im2, ax=ax2)
im3 = ax3.imshow((adj_pred > 0).astype(np.float_))
plt.colorbar(im3, ax=ax3)

## Plot the embeddings

In [ ]:
import pickle

import seaborn as sb

from sklearn.manifold import MDS

In [ ]:
# Compute embeddings
feed_dict = construct_feed_dict(adj_norm, adj_label, features, placeholders)
feed_dict.update({placeholders['dropout']: 0})
emb = sess.run(model.z_mean, feed_dict=feed_dict)

In [ ]:
# Downscale the embeddings
mds = MDS(n_jobs=-2)
mds.fit(emb)

In [ ]:
# Load the ground truth labels

def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

ty = pickle.load(open("data/ind.cora.ty", 'rb'), encoding='latin1')
ally = pickle.load(open("data/ind.cora.ally", 'rb'), encoding='latin1')
test_idx_reorder = parse_index_file("data/ind.cora.test.index")
test_idx_range = np.sort(test_idx_reorder)

labels = sp.vstack((ally, ty)).toarray()
labels[test_idx_reorder, :] = labels[test_idx_range, :]

In [ ]:
# Turn labels into colors
palette = sb.color_palette(n_colors=labels.shape[1])
colors = np.array(palette)[np.argmax(labels, axis=1)]

In [ ]:
# Plot the downscaled embeddings and the links
fig, ax = plt.subplots(figsize=(15, 15))
edges = np.array([[mds.embedding_[i], mds.embedding_[j]] for (i, j) in sparse_to_tuple(sp.triu(adj))[0]])
edges = edges.transpose([2, 1, 0])
ax.plot(edges[0], edges[1], color='lightgrey', zorder=1)
ax.scatter(mds.embedding_[:, 0], mds.embedding_[:, 1], c=colors, zorder=2)